In [6]:
import csv
import math
import pandas as pd
from scipy.stats import poisson
from scipy.stats import laplace
from scipy.stats import gennorm
from diffprivlib.mechanisms import Laplace
import matplotlib.pyplot as plt

In [11]:
df = pd.read_csv('./Sample Data/R01E01D01.csv')

In [12]:
def privatise(column):
    laplace = Laplace()
    if df[column].dtype == 'int64':
        laplace.set_epsilon(1)
        laplace.set_sensitivity(df[column].mean()*0.1)
        df[column] = df[column].apply(laplace.randomise).round(0).astype(int)
    elif df[column].dtype == 'object':
        vals = df[column].unique()
        mapping = dict([(y,x) for x,y in enumerate(sorted(set(vals)))]) 
        rev_map = {k: v for k, v in enumerate(mapping)}
        print(rev_map)
        df[column] = df[column].map(mapping)
        laplace.set_epsilon(1)
        laplace.set_sensitivity(df[column].mean()*0.5)
        df[column] = df[column].apply(laplace.randomise).round(0).astype(int)
        print(rev_map)
        df[column] = df[column].map(rev_map)
    elif df[column].dtype == 'float64':
        laplace.set_epsilon(1)
        laplace.set_sensitivity(df[column].mean()*0.1)
        df[column] = df[column].apply(laplace.randomise)
    else:
        print(column + ' Datatype not obfuscatable')

def get_columns():
    print(df.columns)


get_columns()

Index(['ID', 'Form', '1st reduction', '2nd reduction', 'trial', 'duration',
       '3D deviation', 'XY deviation', 'XY directed', 'Z deviation',
       'Z directed', 'Z std deviation '],
      dtype='object')
